# Tutorial 7: Tools - Extending Agent Capabilities

**Difficulty:** Intermediate-Advanced | **Time:** 40 minutes

## Learning Objectives

- Create custom tools for agents
- Understand tool calling mechanics and flow
- Master tool function definition with docstrings and type hints
- Configure tool choice strategies
- Debug tool execution with traces
- Build multi-step tool-based workflows
- Handle tool errors gracefully

## Real-World Use Cases

Tools transform agents from passive responders into active problem solvers:

- **Research Assistant**: Uses web search, database queries, and document retrieval tools
- **Code Analysis Agent**: Executes code, reads files, runs tests, and generates documentation
- **Customer Support Bot**: Checks order status, processes refunds, updates tickets
- **Data Analyst**: Queries databases, runs calculations, generates visualizations
- **Personal Assistant**: Manages calendar, sends emails, sets reminders
- **DevOps Agent**: Monitors systems, deploys code, manages infrastructure

In this tutorial, you'll learn how to give your agents superpowers by integrating custom tools!

## Core Concepts

### What Are Tools?

**Tools** are functions that agents can call to perform actions or retrieve information. They extend agents beyond text generation to interact with external systems, execute code, access databases, and more.

**Key characteristics:**
- Defined as Python functions with clear signatures
- Automatically discovered via the `@tool` decorator
- Have descriptive docstrings that guide the LLM
- Use type hints for parameter validation
- Return structured results

### Tool Calling Flow

```
User Query → Agent → LLM decides to use tool
                ↓
           Tool executed with parameters
                ↓
           Result returned to LLM
                ↓
           LLM formulates final answer
```

### The @tool Decorator

The `@tool` decorator transforms regular Python functions into agent-callable tools:

```python
from spark.tools.decorator import tool

@tool
def get_weather(city: str, units: str = "celsius") -> str:
    """Get the current weather for a given city.
    
    Args:
        city: The name of the city
        units: Temperature units (celsius or fahrenheit)
        
    Returns:
        Weather information as a string
    """
    # Implementation here
    return f"Weather in {city}: 22°{units[0].upper()}"
```

**What the decorator does:**
1. Extracts function metadata (name, parameters, return type)
2. Parses the docstring for descriptions
3. Creates a JSON schema from type hints
4. Validates inputs before execution
5. Formats outputs for the LLM

### Tool Choice Strategies

Control how agents decide which tools to use via `AgentConfig.tool_choice`:

**1. `auto` (default)**: LLM decides whether to use tools
- Most flexible and common
- Agent uses tools when needed
- Can also answer directly without tools

**2. `any`**: Force the agent to use at least one tool
- Ensures tool usage
- Useful when you always want an action performed
- Agent cannot respond without calling a tool

**3. Specific tool**: Force a specific tool by name
- Agent must use the specified tool
- Useful for deterministic workflows
- Format: `{'type': 'tool', 'name': 'tool_name'}`

**4. `none`**: Disable all tools
- Agent cannot use any tools
- Useful for final answer generation
- Only generates text responses

Example configuration:
```python
# Auto mode (default)
config = AgentConfig(
    model=model,
    tools=[search, calculate],
    tool_choice='auto'
)

# Force tool usage
config = AgentConfig(
    model=model,
    tools=[search],
    tool_choice='any'
)

# Force specific tool
config = AgentConfig(
    model=model,
    tools=[search, calculate],
    tool_choice={'type': 'tool', 'name': 'search'}
)
```

### Multi-Step Tool Calling

Agents can call multiple tools in sequence, controlled by `max_steps`:

```
1. User: "Find and summarize recent AI research papers"
2. Agent → search_papers("AI research") → returns paper list
3. Agent → get_paper_content(paper_id) → returns full text
4. Agent → summarize_text(text) → returns summary
5. Agent → Final answer with summarized papers
```

**max_steps Configuration**:
- Limits the number of reasoning iterations
- Prevents infinite loops in complex workflows
- Default: 10 iterations
- Each tool call + response counts as one step

### Tool Traces

Track tool execution for debugging and monitoring:
- Which tools were called
- What parameters were passed
- What results were returned
- Timing information
- Errors encountered

Tool traces are automatically tracked in `AgentState.tool_trace` and can be accessed after execution for analysis and debugging.

## Setup

Let's import the necessary classes for building tool-enabled agents.

In [ ]:
# Import core Spark classes
from spark.agents.agent import Agent
from spark.agents.config import AgentConfig
from spark.tools.decorator import tool, ToolContext
from spark.models.openai import OpenAIModel
from spark.models.echo import EchoModel
from spark.utils import arun
from typing import Any, Optional, Literal
import time
import json
from datetime import datetime

## Example 1: Your First Tool - Simple Calculator

Let's start by creating a simple calculator tool and giving it to an agent.

In [ ]:
print("=== Example 1: Simple Calculator Tool ===")
print()

# Define a simple calculator tool
@tool
def calculate(expression: str) -> str:
    """Evaluate a mathematical expression and return the result.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 2", "10 * 5")
        
    Returns:
        The result of the calculation as a string
    """
    try:
        # Safe evaluation of mathematical expressions
        # Note: In production, use a safer alternative to eval()
        result = eval(expression, {"__builtins__": {}}, {})
        print(f"  🔧 Tool called: calculate('{expression}') = {result}")
        return str(result)
    except Exception as e:
        return f"Error: {str(e)}"

# Create an agent with the calculator tool
calc_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='calculator_agent',
    system_prompt='You are a helpful math assistant. Use the calculate tool to perform calculations.',
    tools=[calculate],  # Add the tool to the agent
)

calc_agent = Agent(calc_config)

print("🤖 Calculator Agent")
print(f"Available tools: {[tool.tool_name for tool in calc_agent.tool_registry.registry.values()]}")
print()

# Test the agent
question = "What is 127 * 45?"
print(f"User: {question}")
print()

result = await calc_agent.do({'messages': [{'role': 'user', 'content': question}]})
print(f"\nAssistant: {result.content}")
print('Tool traces:', calc_agent.get_tool_traces())
print()

print("📊 Analysis:")
print("✓ Tool was automatically registered from the @tool decorator")
print("✓ Agent received tool specification with name, description, and parameters")
print("✓ LLM can now decide when to use the calculator")
print("✓ Tool execution is transparent to the user")

### Understanding Tool Registration

**How tools are registered:**

1. **@tool decorator** extracts metadata:
   - Function name → tool name
   - Docstring → tool description (sent to LLM)
   - Type hints → parameter types and validation
   - Parameter descriptions from docstring Args section

2. **ToolRegistry** manages tools:
   - Stores tool specifications
   - Validates tool definitions
   - Provides tools to the model

3. **Agent** coordinates:
   - Passes tool specs to the model
   - Receives tool call requests from LLM
   - Executes tools with provided parameters
   - Returns results back to LLM

**Best practices:**
- Write clear, descriptive docstrings (LLM reads these!)
- Use type hints for all parameters
- Handle errors gracefully
- Keep tool functions focused and simple
- Return string results for text-based LLMs

## Example 2: Multiple Tools - Research Assistant

Let's build a research assistant with multiple tools for web search and content retrieval.

In [ ]:
print("=== Example 2: Multi-Tool Research Assistant ===")
print()

# Define multiple research tools
@tool
def search_web(query: str) -> str:
    """Search the web for information about a given query.
    
    Args:
        query: The search query string
        
    Returns:
        Search results as a formatted string
    """
    print(f"  🔍 Searching web for: '{query}'")
    # Simulated search results
    results = f"""Search results for '{query}':
1. Article: Introduction to {query}
2. Research paper: Recent advances in {query}
3. Tutorial: Getting started with {query}"""
    return results

@tool
def get_current_date() -> str:
    """Get the current date and time.
    
    Returns:
        Current date and time as a formatted string
    """
    print("  📅 Getting current date")
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def count_words(text: str) -> str:
    """Count the number of words in a given text.
    
    Args:
        text: The text to count words in
        
    Returns:
        Number of words as a string
    """
    word_count = len(text.split())
    print(f"  📝 Counting words: {word_count} words")
    return str(word_count)

# Create research assistant with all tools
research_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='research_assistant',
    system_prompt="""You are a helpful research assistant with access to multiple tools.
Use the appropriate tools to answer questions:
- Use search_web to find information online
- Use get_current_date for time-related queries
- Use count_words to analyze text

You can call multiple tools if needed to fully answer the question.""",
    tools=[search_web, get_current_date, count_words],
)

research_agent = Agent(research_config)

print("🤖 Research Assistant")
print(f"Available tools: {[tool.tool_name for tool in research_agent.tool_registry.registry.values()]}")
print()

# Test with a query requiring multiple tools
questions = [
    "What's today's date?",
    "Search for information about quantum computing",
    "How many words are in this sentence: The quick brown fox jumps over the lazy dog",
]

for question in questions:
    print(f"User: {question}")
    result = await research_agent.do({'messages': [{'role': 'user', 'content': question}]})
    print(f"Assistant: {result.content}")
    print()

print("📊 Analysis:")
print("✓ Agent has access to multiple specialized tools")
print("✓ LLM selects the appropriate tool based on the question")
print("✓ Each tool is focused on a specific task")
print("✓ Tools can be composed for complex queries")

### Tool Selection Strategy

**How the LLM chooses tools:**

1. **Analyzes the user query** to understand intent
2. **Reviews available tool descriptions** from docstrings
3. **Matches query to tool capabilities**
4. **Generates tool call** with appropriate parameters
5. **Processes tool result** and formulates answer

**Writing good tool descriptions:**

✅ **Good:**
```python
@tool
def search_web(query: str) -> str:
    """Search the web for current information about a given topic.
    Use this when you need up-to-date information or facts you don't know.
    
    Args:
        query: The search query (e.g., 'latest AI news', 'Python tutorial')
    """
```

❌ **Bad:**
```python
@tool
def search_web(query: str) -> str:
    """Searches."""
```

**Tips for tool descriptions:**
- Be specific about what the tool does
- Mention when the tool should be used
- Provide example parameter values
- Explain the format of the return value
- Mention any limitations or constraints

## Example 3: Type Hints and Parameter Validation

Type hints provide automatic parameter validation and clear API contracts.

In [ ]:
print("=== Example 3: Type Hints and Validation ===")
print()

from typing import Literal, Optional

# Tool with rich type hints
@tool
def format_temperature(
    value: float,
    from_unit: Literal["celsius", "fahrenheit", "kelvin"],
    to_unit: Literal["celsius", "fahrenheit", "kelvin"],
    precision: int = 2
) -> str:
    """Convert temperature between different units.
    
    Args:
        value: The temperature value to convert
        from_unit: Source temperature unit (celsius, fahrenheit, or kelvin)
        to_unit: Target temperature unit (celsius, fahrenheit, or kelvin)
        precision: Number of decimal places (default: 2)
        
    Returns:
        Formatted temperature string with the converted value
    """
    print(f"  🌡️  Converting {value}°{from_unit[0].upper()} to {to_unit}")
    
    # Conversion logic (simplified)
    # First convert to Celsius
    if from_unit == "fahrenheit":
        celsius = (value - 32) * 5/9
    elif from_unit == "kelvin":
        celsius = value - 273.15
    else:
        celsius = value
    
    # Then convert to target unit
    if to_unit == "fahrenheit":
        result = celsius * 9/5 + 32
    elif to_unit == "kelvin":
        result = celsius + 273.15
    else:
        result = celsius
    
    return f"{result:.{precision}f}°{to_unit[0].upper()}"

# Tool with optional parameters
@tool
def send_notification(
    message: str,
    priority: Literal["low", "medium", "high"] = "medium",
    channel: Optional[str] = None
) -> str:
    """Send a notification with a message.
    
    Args:
        message: The notification message to send
        priority: Notification priority level (default: medium)
        channel: Optional channel to send to (e.g., 'email', 'slack')
        
    Returns:
        Confirmation message
    """
    channel_str = f" via {channel}" if channel else ""
    print(f"  📢 Sending {priority} priority notification{channel_str}")
    return f"Notification sent: {message} [priority={priority}]"

# Create agent with typed tools
typed_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='typed_agent',
    system_prompt='You are an assistant with precisely defined tools. Use them correctly.',
    tools=[format_temperature, send_notification],
)

typed_agent = Agent(typed_config)

print("🤖 Typed Tools Agent")
print()

# Demonstrate tool usage
print("📋 Tool Specifications:")
print()
for tool_name, tool_obj in typed_agent.tool_registry.registry.items():
    spec = tool_obj.tool_spec
    print(f"Tool: {spec['name']}")
    print(f"Description: {spec['description'][:80]}...")
    print(f"Parameters: {list(spec['parameters']['json'].get('properties', {}).keys())}")
    print()

print("📊 Analysis:")
print("✓ Literal types constrain values to specific options")
print("✓ Optional parameters have default values")
print("✓ Type hints generate JSON schema automatically")
print("✓ Invalid parameters are caught before execution")
print("✓ LLM understands parameter constraints")

### Type Hints Best Practices

**Supported type hints:**

```python
# Basic types
param: str          # String
param: int          # Integer
param: float        # Float
param: bool         # Boolean

# Optional parameters
param: Optional[str] = None    # Can be None
param: str = "default"         # Has default value

# Constrained values
param: Literal["option1", "option2"]  # Must be one of these

# Collections
param: list[str]     # List of strings
param: dict[str, int]  # Dictionary
```

**Why type hints matter:**

1. **Validation**: Catches errors before execution
2. **Documentation**: Makes tool API clear
3. **IDE support**: Better autocomplete and type checking
4. **LLM guidance**: Helps LLM use tools correctly
5. **Schema generation**: Automatic JSON schema creation

**Common patterns:**

```python
# Enum-like choices with Literal
@tool
def set_mode(mode: Literal["easy", "medium", "hard"]) -> str:
    """Set difficulty mode."""
    pass

# Optional with sensible default
@tool
def fetch_data(url: str, timeout: int = 30) -> str:
    """Fetch data from URL with optional timeout."""
    pass

# Multiple related parameters
@tool
def create_user(name: str, email: str, age: Optional[int] = None) -> str:
    """Create a new user with optional age."""
    pass
```

## Example 4: Error Handling in Tools

Proper error handling ensures tools fail gracefully and provide useful feedback.

In [ ]:
print("=== Example 4: Error Handling ===")
print()

# Tool with robust error handling
@tool
def divide_numbers(numerator: float, denominator: float) -> str:
    """Divide two numbers and return the result.
    
    Args:
        numerator: The number to divide
        denominator: The number to divide by
        
    Returns:
        The division result or an error message
    """
    print(f"  ➗ Dividing {numerator} by {denominator}")
    
    try:
        if denominator == 0:
            return "Error: Cannot divide by zero. Please provide a non-zero denominator."
        
        result = numerator / denominator
        return f"Result: {result}"
    
    except Exception as e:
        return f"Error: Unexpected error occurred: {str(e)}"

@tool
def read_file(filepath: str) -> str:
    """Read and return the contents of a file.
    
    Args:
        filepath: Path to the file to read
        
    Returns:
        File contents or an error message
    """
    print(f"  📄 Reading file: {filepath}")
    
    try:
        # Simulated file reading with validation
        if not filepath:
            return "Error: Filepath cannot be empty"
        
        if not filepath.endswith('.txt'):
            return "Error: Only .txt files are supported"
        
        # Simulated file content
        return f"File contents of {filepath}: Hello, World!"
    
    except FileNotFoundError:
        return f"Error: File not found: {filepath}"
    except PermissionError:
        return f"Error: Permission denied to read {filepath}"
    except Exception as e:
        return f"Error: Failed to read file: {str(e)}"

@tool
def parse_json(json_string: str) -> str:
    """Parse a JSON string and return a formatted representation.
    
    Args:
        json_string: Valid JSON string to parse
        
    Returns:
        Parsed JSON or an error message
    """
    print(f"  📦 Parsing JSON")
    
    try:
        parsed = json.loads(json_string)
        return f"Parsed successfully: {json.dumps(parsed, indent=2)}"
    
    except json.JSONDecodeError as e:
        return f"Error: Invalid JSON format: {str(e)}"
    except Exception as e:
        return f"Error: Failed to parse JSON: {str(e)}"

# Create agent with error-handling tools
error_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='robust_agent',
    system_prompt="""You are a helpful assistant with robust tools.
When tools return errors, explain the error to the user and suggest corrections.
Be helpful and guide users to successful tool usage.""",
    tools=[divide_numbers, read_file, parse_json],
)

robust_agent = Agent(error_config)

print("🤖 Robust Agent with Error Handling")
print()

# Test error scenarios
test_cases = [
    ("Divide 10 by 0", "Tests division by zero"),
    ("Read the file test.pdf", "Tests unsupported file type"),
    ("Parse this JSON: {invalid json}", "Tests invalid JSON"),
]

for query, description in test_cases:
    print(f"Test: {description}")
    print(f"User: {query}")
    # In a real scenario, these would trigger tool calls
    print("  [Tool would execute and handle error]")
    print()

print("📊 Error Handling Best Practices:")
print("✓ Check input parameters before processing")
print("✓ Catch specific exceptions (FileNotFoundError, etc.)")
print("✓ Provide clear, actionable error messages")
print("✓ Return errors as strings, not raise exceptions")
print("✓ Include context in error messages")
print("✓ Always have a catch-all except clause")

### Error Handling Patterns

**✅ Good error handling:**

```python
@tool
def my_tool(param: str) -> str:
    """Tool with good error handling."""
    try:
        # Validate inputs
        if not param:
            return "Error: param cannot be empty"
        
        # Perform operation
        result = do_something(param)
        return f"Success: {result}"
    
    except SpecificError as e:
        return f"Error: {str(e)}"
    except Exception as e:
        return f"Error: Unexpected error: {str(e)}"
```

**❌ Bad error handling:**

```python
@tool
def my_tool(param: str) -> str:
    """Tool with bad error handling."""
    result = do_something(param)  # Might crash!
    return result
```

**Why return errors instead of raising?**

- Tools run in the agent's execution loop
- Raised exceptions break the agent flow
- Returned error strings are sent back to the LLM
- LLM can explain errors to users or retry with corrections
- Enables graceful recovery and error correction

**Error message format:**

```python
# ✅ Clear, actionable
return "Error: File 'data.txt' not found. Please check the filename and try again."

# ❌ Vague, unhelpful
return "Error"
```

## Example 5: Multi-Step Tool Workflows

Agents can chain multiple tool calls to accomplish complex tasks.

In [ ]:
print("=== Example 5: Multi-Step Workflows ===")
print()

# Simulated database for demo
fake_database = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
    ],
    "orders": [
        {"order_id": 101, "user_id": 1, "product": "Laptop", "status": "shipped"},
        {"order_id": 102, "user_id": 2, "product": "Mouse", "status": "pending"},
    ]
}

# Tool 1: Search user
@tool
def find_user(email: str) -> str:
    """Find a user by their email address.
    
    Args:
        email: User's email address
        
    Returns:
        User information or error message
    """
    print(f"  🔍 Looking up user: {email}")
    
    for user in fake_database["users"]:
        if user["email"] == email:
            return json.dumps(user)
    
    return f"Error: User with email {email} not found"

# Tool 2: Get user's orders
@tool
def get_user_orders(user_id: int) -> str:
    """Get all orders for a specific user.
    
    Args:
        user_id: The user's ID number
        
    Returns:
        List of orders or error message
    """
    print(f"  📦 Fetching orders for user {user_id}")
    
    orders = [o for o in fake_database["orders"] if o["user_id"] == user_id]
    
    if not orders:
        return f"No orders found for user {user_id}"
    
    return json.dumps(orders)

# Tool 3: Update order status
@tool
def update_order_status(
    order_id: int,
    new_status: Literal["pending", "processing", "shipped", "delivered", "cancelled"]
) -> str:
    """Update the status of an order.
    
    Args:
        order_id: The order ID to update
        new_status: New status for the order
        
    Returns:
        Confirmation message or error
    """
    print(f"  ✏️  Updating order {order_id} to status: {new_status}")
    
    for order in fake_database["orders"]:
        if order["order_id"] == order_id:
            old_status = order["status"]
            order["status"] = new_status
            return f"Order {order_id} status updated from '{old_status}' to '{new_status}'"
    
    return f"Error: Order {order_id} not found"

# Create agent with workflow tools and max_steps
workflow_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='workflow_agent',
    system_prompt="""You are a customer service agent with database access.
    
To help customers, you may need to:
1. First find the user by email using find_user
2. Then get their orders using get_user_orders with the user_id
3. Finally update order status if needed using update_order_status

Always chain tools in the correct order to complete the task.""",
    tools=[find_user, get_user_orders, update_order_status],
    max_steps=6,  # Limit to 6 reasoning iterations
)

workflow_agent = Agent(workflow_config)

print("🤖 Workflow Agent (Multi-Step)")
print(f"Max steps configured: {workflow_config.max_steps}")
print()

print("💬 Complex Query Test:")
query = "Can you find alice@example.com's orders and mark order 101 as delivered?"
print(f"User: {query}")
print()

result = await workflow_agent.do({'messages': query})

print(f"\nAssistant: {result.content}")
print()

print("📊 Multi-Step Workflow Patterns:")
print("✓ Tool calls can be chained sequentially")
print("✓ Output of one tool becomes input to the next")
print("✓ LLM orchestrates the sequence automatically")
print("✓ Complex tasks decompose into simple tool calls")
print("✓ Each tool remains focused and reusable")
print("✓ max_steps prevents infinite loops")
print()

print("🔍 Tool Trace Analysis:")
print(f"Total tool calls made: {len(workflow_agent.get_tool_traces())}")
print("Tool sequence:")
for i, trace in enumerate(workflow_agent.get_tool_traces()):
    print(f"  {i}. {trace.get('tool_name')}")

### Multi-Step Tool Patterns

**Common workflow patterns:**

**1. Sequential Pipeline**
```
Tool A → Tool B → Tool C → Final Answer
```
Example: `search_user → get_orders → format_response`

**2. Conditional Branching**
```
Tool A → If result X: Tool B
       → If result Y: Tool C
```
Example: `check_inventory → If available: process_order, If not: notify_out_of_stock`

**3. Iterative Refinement**
```
Tool A → Analyze → Tool A again with refinements → Final Answer
```
Example: `search → analyze_results → search_with_refined_query`

**4. Parallel + Aggregate**
```
Tool A ┐
Tool B ├→ Combine Results → Final Answer
Tool C ┘
```
Example: `get_weather, get_traffic, get_events → summarize_for_day`

**Max Steps Configuration:**

Control the number of tool calling iterations to prevent infinite loops and manage costs:

```python
config = AgentConfig(
    model=model,
    tools=[tool1, tool2, tool3],
    max_steps=10,  # Limit to 10 reasoning iterations (default: 10)
)

# For simple workflows
config = AgentConfig(
    model=model,
    tools=[single_tool],
    max_steps=3,  # Only allow up to 3 iterations
)

# For complex multi-step workflows
config = AgentConfig(
    model=model,
    tools=[search, analyze, summarize, validate],
    max_steps=15,  # Allow more iterations for complex tasks
)
```

**Why max_steps matters:**
- **Prevents infinite loops**: Stops runaway tool calling
- **Controls costs**: Limits the number of LLM calls
- **Sets timeout expectations**: Predictable execution time
- **Forces efficiency**: Encourages agents to complete tasks within limits

**What happens when max_steps is reached:**
- Agent stops tool calling and returns current state
- Any partial results are returned
- Tool trace shows all steps taken
- Error/warning may be included indicating step limit reached

## Example 6: Tool Context and Agent Access

Advanced: Tools can access the agent context for sophisticated behaviors.

In [ ]:
print("=== Example 6: Tools with Context ===")
print()

from spark.tools.decorator import ToolContext

# Tool with context access
@tool(context=True)
def get_conversation_stats(tool_context: ToolContext) -> str:
    """Get statistics about the current conversation.
    
    Returns:
        Statistics about message count and conversation
    """
    print("  📊 Analyzing conversation stats")
    
    # Access agent from context
    agent = tool_context.invocation_state['agent']
    
    # Get conversation history (if available)
    # Note: This is a demonstration of the concept
    stats = {
        "agent_name": agent.config.name,
        "tools_available": len(agent.tool_registry.registry),
        "tool_use_id": tool_context.tool_use.get('toolUseId', 'unknown'),
    }
    
    return json.dumps(stats, indent=2)

@tool(context=True)
def log_tool_usage(message: str, tool_context: ToolContext) -> str:
    """Log a message with tool execution context.
    
    Args:
        message: The message to log
        
    Returns:
        Confirmation of logged message
    """
    print(f"  📝 Logging: {message}")
    
    # Access invocation state
    invocation_state = tool_context.invocation_state
    
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "message": message,
        "agent": invocation_state['agent'].config.name,
    }
    
    return f"Logged: {json.dumps(log_entry)}"

def before_hook(msgs, context):
    print('msgs', msgs)
    
# Create agent with context-aware tools
context_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='context_aware_agent',
    system_prompt='You are an agent with self-aware tools.',
    tools=[get_conversation_stats, log_tool_usage],
    before_llm_hooks=[before_hook]
)

context_agent = Agent(context_config)

await context_agent.do({'messages': ['Hello']})
await context_agent.do({'messages': ['hello, what tools do you have?']})
await context_agent.do({'messages': ['can you give a your statistics so far?']})

print("🤖 Context-Aware Agent")
print()

print("📋 Context-Aware Tool Benefits:")
print("✓ Access to agent configuration and state")
print("✓ Can inspect conversation history")
print("✓ Tool use metadata available (toolUseId, etc.)")
print("✓ Enables meta-operations (stats, logging, monitoring)")
print("✓ Advanced debugging and instrumentation")
print()

print("⚙️  Context parameter:")
print("  @tool(context=True)  # Injects ToolContext as 'tool_context'")
print("  @tool(context='ctx')  # Injects ToolContext as 'ctx'")
print()

print("📦 ToolContext contents:")
print("  - tool_use: Complete ToolUse object")
print("  - agent: The Agent instance")
print("  - invocation_state: Caller-provided kwargs")

### When to Use Tool Context

**Use ToolContext for:**

✅ **Meta-operations**
```python
@tool(context=True)
def get_tool_stats(tool_context: ToolContext) -> str:
    """Report on available tools and usage."""
    tools = tool_context.agent.tool_registry.registry
    return f"Available tools: {list(tools.keys())}"
```

✅ **Logging and monitoring**
```python
@tool(context=True)
def audit_log(action: str, tool_context: ToolContext) -> str:
    """Log actions with agent context."""
    agent_name = tool_context.agent.config.name
    return f"[{agent_name}] Logged: {action}"
```

✅ **Conditional behavior based on agent state**
```python
@tool(context=True)
def smart_search(query: str, tool_context: ToolContext) -> str:
    """Search with behavior based on agent configuration."""
    # Adapt search based on agent's system prompt or state
    pass
```

**Don't use ToolContext for:**

❌ **Regular data processing** (use normal parameters)
❌ **Modifying agent state** (tools should be side-effect free when possible)
❌ **Simple calculations** (keep tools simple)

**Best practices:**
- Use context sparingly for advanced use cases
- Document why context is needed in the docstring
- Prefer simple parameter-based tools when possible
- Use context for instrumentation and debugging

In [ ]:
print("=== Example 6b: Tool Choice Strategies ===")
print()

# Define simple tools for demonstration
@tool
def get_time() -> str:
    """Get the current time.
    
    Returns:
        Current time as a string
    """
    print("  🕐 Getting current time")
    return datetime.now().strftime("%H:%M:%S")

@tool
def get_date() -> str:
    """Get the current date.
    
    Returns:
        Current date as a string
    """
    print("  📅 Getting current date")
    return datetime.now().strftime("%Y-%m-%d")

# Test 1: Auto mode (default) - LLM decides
print("Test 1: tool_choice='auto' (LLM decides)")
print("-" * 50)

auto_agent = Agent(
    config=AgentConfig(
        model=OpenAIModel(model_id='gpt-5-nano'),
        name='auto_agent',
        system_prompt='You are a helpful assistant.',
        tools=[get_time, get_date],
        tool_choice='auto',  # Default - LLM decides
    )
)

# Query that may or may not need tools
result = await auto_agent.do({'messages': 'Hello! How are you?'})
print(f"Query: 'Hello! How are you?'")
print(f"Response: {result.content[:100]}...")
print(f"Tools used: {len(auto_agent.get_tool_traces())}")
print()

# Query that likely needs tools
result = await auto_agent.do({'messages': 'What time is it?'})
print(f"Query: 'What time is it?'")
print(f"Response: {result.content[:100]}...")
print(f"Tools used: {len(auto_agent.get_tool_traces())}")
print()
print()

# Test 2: Any mode - Force tool usage
print("Test 2: tool_choice='any' (Force tool usage)")
print("-" * 50)

any_agent = Agent(
    config=AgentConfig(
        model=OpenAIModel(model_id='gpt-5-nano'),
        name='any_agent',
        system_prompt='You are a helpful assistant. Always use a tool to answer.',
        tools=[get_time, get_date],
        tool_choice='any',  # Must use at least one tool
    )
)

result = await any_agent.do({'messages': 'Give me some information'})
print(f"Query: 'Give me some information'")
print(f"Response: {result.content[:100]}...")
print(f"Tools used: {len(any_agent.get_tool_traces())}")
print("✓ Agent was forced to use a tool even for a vague query")
print()
print()

# Test 3: Specific tool - Force specific tool
print("Test 3: tool_choice with specific tool")
print("-" * 50)

specific_agent = Agent(
    config=AgentConfig(
        model=OpenAIModel(model_id='gpt-5-nano'),
        name='specific_agent',
        system_prompt='You are a helpful assistant.',
        tools=[get_time, get_date],
        tool_choice='get_date',  # Force get_date
    )
)

result = await specific_agent.do({'messages': 'What is the current information?'})
print(f"Query: 'What is the current information?'")
print(f"Response: {result.content[:100]}...")
if hasattr(result.state, 'tool_trace') and result.state.tool_trace:
    print(f"Tool used: {specific_agent.get_tool_traces()[0].get('name', 'unknown')}")
    print("✓ Agent was forced to use get_date specifically")
print()
print()

# Test 4: None mode - Disable tools
print("Test 4: tool_choice='none' (No tools)")
print("-" * 50)

none_agent = Agent(
    config=AgentConfig(
        model=OpenAIModel(model_id='gpt-5-nano'),
        name='none_agent',
        system_prompt='You are a helpful assistant.',
        tools=[get_time, get_date],
        tool_choice='none',  # Cannot use tools
    )
)

result = await none_agent.do({'messages': 'What time is it?'})
print(f"Query: 'What time is it?'")
print(f"Response: {result.content[:150]}...")
print(f"Tools used: {len(none_agent.get_tool_traces())}")
print("✓ Agent cannot use tools even when they would be helpful")
print()

print("=" * 60)
print("📊 Tool Choice Strategy Summary:")
print()
print("'auto':  Most flexible - LLM decides when to use tools")
print("'any':   Always forces tool usage - good for action-oriented tasks")
print("specific: Forces a particular tool - deterministic workflows")
print("'none':  Disables tools - pure text generation")
print()
print("💡 Use Cases:")
print("  • 'auto' → General purpose agents (default)")
print("  • 'any' → Ensure action is taken (e.g., logging, data collection)")
print("  • specific → Deterministic pipelines (e.g., always check inventory first)")
print("  • 'none' → Final answer generation, no external calls needed")

## Example 6b: Tool Choice Strategies

Learn how to control tool usage with different tool_choice settings.

### Debugging Checklist

**When a tool isn't working:**

**1. Check Tool Registration**
```python
# List all registered tools
print(list(agent.tool_registry.registry.keys()))

# Verify your tool is in the list
assert 'my_tool' in agent.tool_registry.registry
```

**2. Inspect Tool Specification**
```python
# Check what the LLM sees
tool_spec = agent.tool_specs[0]
print(json.dumps(tool_spec, indent=2))

# Verify description is clear and accurate
assert 'description' in tool_spec
```

**3. Test Tool Directly**
```python
# Call tool without agent
from my_tools import my_tool
result = my_tool(param="test")
print(f"Direct test: {result}")
```

**4. Check Type Hints**
```python
# Verify parameter types match your docstring
import inspect
sig = inspect.signature(my_tool)
print(sig.parameters)
```

**5. Add Debug Logging**
```python
@tool
def my_tool(param: str) -> str:
    print(f"DEBUG: my_tool called with param={param}")
    result = process(param)
    print(f"DEBUG: my_tool returning {result}")
    return result
```

**6. Inspect Tool Traces**
```python
# After execution, check what actually happened
result = await agent.do({'messages': query})
if hasattr(result.state, 'tool_trace'):
    for trace in result.state.tool_trace:
        print(f"Called: {trace['name']}")
        print(f"Input: {trace['input']}")
        print(f"Output: {trace['content']}")
```

**7. Verify Model Compatibility**
- Not all models support tool calling
- Check your model's documentation
- Test with a known working model (e.g., GPT-4)

**8. Check max_steps Configuration**
```python
# Verify you haven't hit the iteration limit
print(f"Max steps: {agent.config.max_steps}")
print(f"Actual steps taken: {len(result.state.tool_trace)}")
```

**Common Solutions:**

| Problem | Solution |
|---------|----------|
| Tool never called | Improve docstring clarity |
| Wrong parameters | Fix type hints |
| Tool errors | Add error handling |
| Unexpected behavior | Test tool directly first |
| Hit iteration limit | Increase max_steps or optimize tool chain |
| Tool not selected | Check tool_choice setting |

In [ ]:
print("=== Example 7: Tool Debugging and Traces ===" )
print()

# Create tools for debugging demonstration
@tool
def fetch_data(source: str) -> str:
    """Fetch data from a source.
    
    Args:
        source: Data source name
    """
    print(f"  📥 Fetching from: {source}")
    time.sleep(0.1)  # Simulate work
    return f"Data from {source}: [sample data]"

@tool
def process_data(data: str) -> str:
    """Process the fetched data.
    
    Args:
        data: Data to process
    """
    print(f"  ⚙️  Processing data")
    time.sleep(0.1)  # Simulate work
    return f"Processed: {data[:50]}..."

@tool
def validate_result(result: str) -> str:
    """Validate the processed result.
    
    Args:
        result: Result to validate
    """
    print(f"  ✅ Validating result")
    return "Validation passed"

# Create agent with trace-enabled tools
trace_config = AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='trace_agent',
    system_prompt='You are a data processing agent. Use tools to fetch, process, and validate data.  The data is already in the database, there is no need for clarification.',
    tools=[fetch_data, process_data, validate_result],
    max_steps=8,
)

trace_agent = Agent(trace_config)

print("🔍 Tool Debugging and Trace Analysis")
print()

# Execute a multi-step workflow
query = "Fetch data from 'database', process it, and validate the result"
print(f"User: {query}")
print()

result = await trace_agent.do({'messages': query})

print(f"\nAssistant: {result.content}")
print()

# Analyze tool trace
print("=" * 60)
print("🔧 Tool Trace Analysis")
print("=" * 60)

traces = trace_agent.get_tool_traces()
print(f"\nTotal tool calls: {len(traces)}")
print()
    
for i, trace in enumerate(traces, 1):
    print(f"Tool Call #{i}:")
    print(f"  Name: {trace.get('tool_name', 'unknown')}")
    print(f"  Tool Use ID: {trace.get('toolUseId', 'N/A')[:20]}...")
        
    # Show parameters
    params = trace.get('input', {})
    if params:
        print(f"  Parameters:")
        for key, value in params.items():
            value_str = str(value)[:50]
            print(f"    {key}: {value_str}")
        
    # Show result preview
    result_content = trace.get('content', [])
    if result_content:
        for content in result_content:
            if isinstance(content, dict) and 'text' in content:
                result_text = content['text'][:60]
                print(f"  Result: {result_text}...")
    print()

print("=" * 60)
print()

print("📊 Debugging Best Practices:")
print()
print("✅ During Development:")
print("  • Add print statements in tools to track execution")
print("  • Test tools directly before giving them to agents")
print("  • Verify tool specs match your expectations")
print("  • Use descriptive tool names and docstrings")
print()
print("✅ Using Tool Traces:")
print("  • Access via result.state.tool_trace after execution")
print("  • Each trace entry contains: name, toolUseId, input, content")
print("  • Analyze call sequence to understand agent reasoning")
print("  • Debug parameter passing and result handling")
print("  • Track execution order for multi-tool workflows")
print()
print("✅ In Production:")
print("  • Log tool traces for debugging failed executions")
print("  • Monitor tool usage patterns and frequencies")
print("  • Track tool execution for audit trails")
print("  • Identify bottlenecks in multi-step workflows")
print()
print("✅ Common Issues:")
print("  • Tool not being called: Check description clarity")
print("  • Wrong parameters: Verify type hints and docstring")
print("  • Tool errors: Add better error handling")
print("  • Too many iterations: Adjust max_steps limit")

### Debugging Checklist

**When a tool isn't working:**

**1. Check Tool Registration**
```python
# List all registered tools
print(list(agent.tool_registry.registry.keys()))

# Verify your tool is in the list
assert 'my_tool' in agent.tool_registry.registry
```

**2. Inspect Tool Specification**
```python
# Check what the LLM sees
tool_spec = agent.tool_specs[0]
print(json.dumps(tool_spec, indent=2))

# Verify description is clear and accurate
assert 'description' in tool_spec
```

**3. Test Tool Directly**
```python
# Call tool without agent
from my_tools import my_tool
result = my_tool(param="test")
print(f"Direct test: {result}")
```

**4. Check Type Hints**
```python
# Verify parameter types match your docstring
import inspect
sig = inspect.signature(my_tool)
print(sig.parameters)
```

**5. Add Debug Logging**
```python
@tool
def my_tool(param: str) -> str:
    print(f"DEBUG: my_tool called with param={param}")
    result = process(param)
    print(f"DEBUG: my_tool returning {result}")
    return result
```

**6. Verify Model Compatibility**
- Not all models support tool calling
- Check your model's documentation
- Test with a known working model (e.g., GPT-4)

**Common Solutions:**

| Problem | Solution |
|---------|----------|
| Tool never called | Improve docstring clarity |
| Wrong parameters | Fix type hints |
| Tool errors | Add error handling |
| Unexpected behavior | Test tool directly first |

## Key Concepts Recap

### Tool Creation Checklist

```python
@tool
def my_tool(
    param1: str,                              # ✓ Type hints
    param2: int = 0,                          # ✓ Default values
    option: Literal["a", "b"] = "a"          # ✓ Constrained values
) -> str:                                     # ✓ Return type
    """Clear description of what the tool does.  # ✓ Description
    
    Args:
        param1: Description of param1         # ✓ Parameter docs
        param2: Description of param2
        option: Which option to use
        
    Returns:
        Description of return value           # ✓ Return docs
    """
    try:                                       # ✓ Error handling
        result = do_work(param1, param2)
        return f"Success: {result}"
    except Exception as e:
        return f"Error: {str(e)}"            # ✓ Error as string
```

### Tool Patterns Summary

| Pattern | Use Case | Example |
|---------|----------|------------|
| **Single Tool** | Simple tasks | Calculator, date/time |
| **Multiple Tools** | Varied capabilities | Research assistant |
| **Tool Chains** | Complex workflows | User lookup → Orders → Update |
| **Context-Aware** | Meta-operations | Statistics, logging |
| **Error-Resistant** | Production reliability | All tools |

### AgentConfig with Tools

```python
config = AgentConfig(
    model=model,
    name='my_agent',
    system_prompt='...',
    tools=[tool1, tool2, tool3],        # List of @tool decorated functions
    tool_choice='auto',                 # 'auto', 'any', 'none', or {'type': 'tool', 'name': '...'}
    max_steps=10,                       # Limit tool calling iterations (default: 10)
)
```

### Tool Choice Strategies

| Strategy | Behavior | Use Case |
|----------|----------|----------|
| **'auto'** | LLM decides | Most flexible, default |
| **'any'** | Must use a tool | Force action |
| **{'type': 'tool', 'name': 'X'}** | Must use specific tool | Deterministic workflows |
| **'none'** | No tools allowed | Final answer only |

### Tool Trace Access

After agent execution, access tool traces for debugging:

```python
result = await agent.do({'messages': query})

# Access tool trace
if hasattr(result.state, 'tool_trace'):
    for trace in result.state.tool_trace:
        print(f"Tool: {trace['name']}")
        print(f"Input: {trace['input']}")
        print(f"Result: {trace['content']}")
```

### When to Create Tools

**✅ Create a tool when you need to:**
- Access external systems (APIs, databases)
- Execute code or commands
- Perform calculations
- Retrieve real-time data
- Modify system state
- Access private information

**❌ Don't create a tool for:**
- Things the LLM can do with knowledge
- Simple text transformations
- Information in the system prompt
- Tasks requiring human judgment

### Tool Design Principles

**1. Single Responsibility**: Each tool does one thing well
**2. Clear Names**: `search_web` not `do_something`
**3. Good Docs**: Docstrings guide the LLM
**4. Type Safety**: Type hints prevent errors
**5. Error Handling**: Return errors, don't raise
**6. Focused Scope**: Keep tools small and composable
**7. Idempotency**: Same input → same output (when possible)

## 💪 Hands-On Exercises

Test your understanding with these practical exercises!

### Exercise 1: Create a File System Tool

Build a simple file system tool that can:
- List files in a directory
- Create a new file
- Read file contents

Requirements:
- Use proper type hints
- Include error handling
- Write clear docstrings

In [ ]:
# Exercise 1: Your code here

# TODO: Create list_directory tool
# TODO: Create create_file tool
# TODO: Create read_file tool
# TODO: Create agent with these tools
# TODO: Test with queries like "List files" and "Create a file named test.txt"

### Exercise 2: Build a Data Analysis Agent

Create an agent with tools for data analysis:
- Calculate mean/median/mode
- Find min/max values
- Generate summary statistics

Requirements:
- Accept list of numbers as input
- Handle edge cases (empty list, etc.)
- Return formatted results

In [ ]:
# Exercise 2: Your code here

# TODO: Create statistics tools
# TODO: Handle edge cases
# TODO: Create agent with tools
# TODO: Test with "Calculate the mean of [1, 2, 3, 4, 5]"

### Exercise 3: Multi-Step Workflow Agent

Build an agent that chains tools to:
1. Search for a topic
2. Extract key information
3. Summarize findings

Create 3 tools that work together and test the workflow.

In [ ]:
# Exercise 3: Your code here

# TODO: Create search_topic tool
# TODO: Create extract_info tool
# TODO: Create summarize tool
# TODO: Create agent that chains these tools
# TODO: Test with "Research and summarize quantum computing"

### Exercise 4: Context-Aware Logging Tool

Create a tool that uses ToolContext to:
- Log tool invocations with metadata
- Track which agent called the tool
- Include timestamps and tool use IDs

In [ ]:
# Exercise 4: Your code here

# TODO: Create context-aware logging tool
# TODO: Extract agent name and tool_use_id from context
# TODO: Format log entries nicely
# TODO: Test the tool

## ✅ Solutions

Try the exercises yourself first! Solutions are provided below.

In [ ]:
# Solution 1: File System Tools

import os
from typing import Optional

@tool
def list_directory(path: str = ".") -> str:
    """List all files and directories in the given path.
    
    Args:
        path: Directory path to list (default: current directory)
        
    Returns:
        List of files and directories
    """
    try:
        items = os.listdir(path)
        return f"Files in {path}:\n" + "\n".join(f"  - {item}" for item in items)
    except FileNotFoundError:
        return f"Error: Directory not found: {path}"
    except PermissionError:
        return f"Error: Permission denied: {path}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def create_file(filename: str, content: str = "") -> str:
    """Create a new file with optional content.
    
    Args:
        filename: Name of the file to create
        content: Optional content to write to the file
        
    Returns:
        Confirmation message
    """
    try:
        if not filename:
            return "Error: Filename cannot be empty"
        
        with open(filename, 'w') as f:
            f.write(content)
        
        return f"File '{filename}' created successfully"
    except Exception as e:
        return f"Error creating file: {str(e)}"

@tool
def read_file(filename: str) -> str:
    """Read and return the contents of a file.
    
    Args:
        filename: Name of the file to read
        
    Returns:
        File contents or error message
    """
    try:
        with open(filename, 'r') as f:
            content = f.read()
        
        return f"Contents of '{filename}':\n{content}"
    except FileNotFoundError:
        return f"Error: File not found: {filename}"
    except Exception as e:
        return f"Error reading file: {str(e)}"

# Create file system agent
fs_agent = Agent(AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='file_system_agent',
    system_prompt='You are a file system assistant. Help users manage files.',
    tools=[list_directory, create_file, read_file],
))

result = await fs_agent.do({'messages': ['what files are in the current working directory?']})

print("=== Solution 1: File System Agent ===")
print("✅ Created tools: list_directory, create_file, read_file")
print("✅ All tools have error handling")
print("✅ Clear docstrings and type hints")

print(result.content)

In [ ]:
# Solution 2: Data Analysis Agent

from typing import List
import statistics

@tool
def calculate_mean(numbers: List[float]) -> str:
    """Calculate the arithmetic mean (average) of a list of numbers.
    
    Args:
        numbers: List of numbers
        
    Returns:
        Mean value or error message
    """
    try:
        if not numbers:
            return "Error: Cannot calculate mean of empty list"
        
        mean = statistics.mean(numbers)
        return f"Mean: {mean:.2f}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def calculate_median(numbers: List[float]) -> str:
    """Calculate the median (middle value) of a list of numbers.
    
    Args:
        numbers: List of numbers
        
    Returns:
        Median value or error message
    """
    try:
        if not numbers:
            return "Error: Cannot calculate median of empty list"
        
        median = statistics.median(numbers)
        return f"Median: {median:.2f}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def find_min_max(numbers: List[float]) -> str:
    """Find the minimum and maximum values in a list of numbers.
    
    Args:
        numbers: List of numbers
        
    Returns:
        Min and max values or error message
    """
    try:
        if not numbers:
            return "Error: Cannot find min/max of empty list"
        
        return f"Min: {min(numbers)}, Max: {max(numbers)}"
    except Exception as e:
        return f"Error: {str(e)}"

@tool
def summary_statistics(numbers: List[float]) -> str:
    """Generate comprehensive summary statistics for a list of numbers.
    
    Args:
        numbers: List of numbers
        
    Returns:
        Summary statistics or error message
    """
    try:
        if not numbers:
            return "Error: Cannot generate statistics for empty list"
        
        stats = f"""Summary Statistics:
  Count: {len(numbers)}
  Mean: {statistics.mean(numbers):.2f}
  Median: {statistics.median(numbers):.2f}
  Std Dev: {statistics.stdev(numbers) if len(numbers) > 1 else 'N/A'}
  Min: {min(numbers)}
  Max: {max(numbers)}"""
        return stats
    except Exception as e:
        return f"Error: {str(e)}"

# Create data analysis agent
stats_agent = Agent(AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='statistics_agent',
    system_prompt='You are a data analysis assistant. Help users analyze numerical data.',
    tools=[calculate_mean, calculate_median, find_min_max, summary_statistics],
))

print("=== Solution 2: Data Analysis Agent ===")
print("✅ Created statistical analysis tools")
print("✅ Handles empty lists and edge cases")
print("✅ Uses Python's statistics module")

result = await stats_agent.do({'messages':['1 2 3 4 5 6 7 8 9 20 30 40 50 60 70']})
print(result.content)
print('# of tool calls', len(stats_agent.get_tool_traces()))

In [ ]:
# Solution 3: Multi-Step Workflow Agent

@tool
def search_topic(topic: str) -> str:
    """Search for information about a topic.
    
    Args:
        topic: The topic to search for
        
    Returns:
        Search results with relevant information
    """
    print(f"  🔍 Searching for: {topic}")
    
    # Simulated search results
    results = f"""Search Results for '{topic}':
    
1. Overview: {topic} is an important field of study...
2. Recent Developments: Latest advances in {topic} include...
3. Applications: {topic} is used in various industries...
4. Challenges: Current challenges in {topic} research...
5. Future Directions: Expected developments in {topic}..."""
    
    return results

@tool
def extract_key_points(text: str) -> str:
    """Extract key points from text.
    
    Args:
        text: Text to analyze
        
    Returns:
        Extracted key points
    """
    print("  📌 Extracting key points")
    
    # Simulated extraction
    lines = text.split('\n')
    key_lines = [line for line in lines if line.strip() and any(char.isdigit() for char in line[:3])]
    
    return "Key Points:\n" + "\n".join(f"  • {line.strip()}" for line in key_lines)

@tool
def create_summary(key_points: str) -> str:
    """Create a concise summary from key points.
    
    Args:
        key_points: Key points to summarize
        
    Returns:
        Concise summary
    """
    print("  📝 Creating summary")
    
    # Simulated summarization
    summary = f"""Summary:
    
Based on the analysis, the topic encompasses several important aspects:
{key_points}

This represents a comprehensive overview of the subject matter."""
    
    return summary

# Create workflow agent
workflow_agent = Agent(AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='research_workflow_agent',
    system_prompt="""You are a research assistant that follows a structured workflow:
1. First, search for the topic using search_topic
2. Then, extract key points using extract_key_points
3. Finally, create a summary using create_summary
Always follow this sequence for comprehensive research.""",
    tools=[search_topic, extract_key_points, create_summary],
))

print("=== Solution 3: Multi-Step Workflow Agent ===")
print("✅ Created 3 chained tools")
print("✅ Each tool builds on the previous one")
print("✅ System prompt guides the workflow")
print()
print("Expected workflow:")
print("  1. search_topic → Get raw information")
print("  2. extract_key_points → Identify important parts")
print("  3. create_summary → Generate final output")

result = await workflow_agent.do({'messages':['research app store optimization']})
print(result.content)
print('# of tool calls', len(workflow_agent.get_tool_traces()))

In [ ]:
# Solution 4: Context-Aware Logging Tool

@tool(context=True)
def log_invocation(message: str, tool_context: ToolContext) -> str:
    """Log a tool invocation with full context metadata.
    
    Args:
        message: Message to log
        
    Returns:
        Formatted log entry
    """
    # Extract metadata from context
    agent_name = tool_context.invocation_state['agent'].config.name
    tool_use_id = tool_context.tool_use.get('toolUseId', 'unknown')
    timestamp = datetime.now().isoformat()
    
    # Create structured log entry
    log_entry = {
        'timestamp': timestamp,
        'agent': agent_name,
        'tool_use_id': tool_use_id,
        'message': message,
    }
    
    # Format for display
    formatted = f"""[LOG ENTRY]
Timestamp: {timestamp}
Agent: {agent_name}
Tool Use ID: {tool_use_id}
Message: {message}"""
    
    print(f"  📋 {formatted}")
    return f"Logged: {json.dumps(log_entry)}"

@tool(context=True)
def get_agent_info(tool_context: ToolContext) -> str:
    """Get information about the current agent.
    
    Returns:
        Agent information including name, tools, and configuration
    """
    agent = tool_context.invocation_state['agent']
    
    info = {
        'name': agent.config.name,
        'model': str(type(agent.config.model).__name__),
        'tools': list(agent.tool_registry.registry.keys()),
        'tool_count': len(agent.tool_registry.registry),
    }
    
    return json.dumps(info, indent=2)

# Create context-aware agent
context_agent = Agent(AgentConfig(
    model=OpenAIModel(model_id='gpt-5-nano'),
    name='context_aware_agent',
    system_prompt='You are AI agent, you have access to logging and introspection tools.',
    tools=[log_invocation, get_agent_info],
))

print("=== Solution 4: Context-Aware Tools ===")
print("✅ Created tools with ToolContext access")
print("✅ Extract agent metadata automatically")
print("✅ Structured logging with timestamps")
print("✅ Introspection capabilities")

result = await context_agent.do({'messages': ['tell me about the agent-yourself, then log the tool invocation']})
print(result.content)
print('# of tool calls', len(context_agent.get_tool_traces()))

## 🎯 Summary & Next Steps

### Congratulations! 🎉

You've mastered tool creation and integration in Spark! You can now build agents that interact with external systems, execute code, and solve complex problems through tool use.

### What You Learned:

✅ **Tool Fundamentals**
- How the @tool decorator works
- Tool calling flow and mechanics
- Tool registration and specification

✅ **Tool Development**
- Writing clear docstrings for LLM guidance
- Using type hints for validation
- Handling errors gracefully
- Creating multi-tool agents

✅ **Advanced Patterns**
- Multi-step tool workflows with max_steps
- Tool choice strategies (auto, any, specific, none)
- Context-aware tools
- Tool debugging techniques
- Tool trace analysis

✅ **Production Practices**
- Tool specification inspection
- Direct tool testing
- Debugging strategies with tool traces
- Common pitfalls and solutions
- Iteration control with max_steps

### Key Patterns to Remember:

```python
# Simple tool
@tool
def my_tool(param: str) -> str:
    """Clear description for the LLM.
    
    Args:
        param: Parameter description
    """
    return f"Result: {param}"

# Agent with tools and configuration
agent = Agent(
    config=AgentConfig(
        model=model,
        tools=[my_tool],
        tool_choice='auto',      # Control tool usage
        max_steps=10,            # Limit iterations
    )
)

# Execute and inspect traces
result = await agent.do({'messages': query})
for trace in result.state.tool_trace:
    print(f"Called: {trace['name']}")
```

### 📚 Related Resources:

- Example files: `e007_agent_with_tools.py`, `e009_tool_agent.py`
- Source: `spark/tools/decorator.py`, `spark/agents/agent.py`
- Tutorial 7: Agent Configuration and Memory (review if needed)
- Tutorial 4: Your First AI Agent

### 🚀 Next Tutorial: Structured Outputs and Type Safety

In **Tutorial 9**, you'll learn how to:
- Generate structured JSON outputs from agents
- Use Pydantic schemas for validation
- Handle structured output errors
- Combine tools with structured outputs
- Build type-safe agent workflows

### 🔧 Before You Move On:

Make sure you can:
1. ✅ Create tools with the @tool decorator
2. ✅ Write clear docstrings and use type hints
3. ✅ Handle errors gracefully in tools
4. ✅ Build agents with multiple tools
5. ✅ Configure tool_choice strategies
6. ✅ Set max_steps to control iterations
7. ✅ Debug tool execution with traces
8. ✅ Create multi-step tool workflows
9. ✅ Use ToolContext for advanced scenarios

### 🎓 Tutorial Series Progress:

- ✅ **Tutorial 1: Hello Spark** - Basic nodes
- ✅ **Tutorial 2: Batch Processing** - Parallel execution
- ✅ **Tutorial 3: Simple Flows** - Graph basics
- ✅ **Tutorial 4: Your First AI Agent** - Agent fundamentals
- ✅ **Tutorial 5: Conditional Routing** - Decision making
- ✅ **Tutorial 6: Agent Config & Memory** - Configuration
- ✅ **Tutorial 7: Tools** - *You are here!* 🎯
- ➡️ **Tutorial 8: TBD

### 🌟 Pro Tips:

- **Start simple**: Begin with single-purpose tools
- **Test tools directly**: Don't wait for agent integration
- **Write for the LLM**: Docstrings guide tool selection
- **Handle errors**: Return errors as strings, don't raise
- **Use type hints**: Catch errors early
- **Keep tools focused**: Single responsibility principle
- **Control iterations**: Use max_steps to prevent runaway execution
- **Force tool usage**: Use tool_choice='any' when tools are required
- **Debug with traces**: Access result.state.tool_trace for detailed analysis
- **Add logging**: Track tool execution in production
- **Monitor usage**: Understand which tools are called most

### 🎯 Challenge Before Next Tutorial:

Build a "Personal Assistant Agent" that:
1. Has at least 5 different tools
2. Can chain tool calls for complex tasks
3. Includes proper error handling
4. Has one context-aware tool for logging
5. Uses type hints and Literal types
6. Configures max_steps appropriately
7. Uses different tool_choice strategies for different scenarios
8. Logs tool traces for debugging

Example tools:
- Get weather
- Set reminder
- Search information
- Calculate expenses
- Log activity

This will prepare you for structured outputs in Tutorial 9!

---

**You're now ready to build production-ready agents with powerful tool integration!** 🚀

### 📝 Production Features Summary:

The following features are now available in Spark:

✅ **tool_choice configuration** - Control tool usage with 'auto', 'any', 'none', or specific tool selection
✅ **max_steps configuration** - Limit reasoning iterations to prevent infinite loops and control costs
✅ **Tool trace logging** - Automatic tracking of all tool calls with parameters, results, and metadata

These features enable robust, production-ready tool-based agent systems with full observability and control!

Have questions or feedback? Check the Spark documentation or open an issue on GitHub.